nothing

In [40]:
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("EDA").getOrCreate()

# Kiểm tra số lượng cores
cores = spark.sparkContext.getConf().get("spark.executor.instances", "1")
print(f"You are working with {cores} core(s).")

# Kiểm tra SparkSession
spark

In [41]:
# Load dữ liệu
path = "C:/Users/vmcch/BIGDATA/sentiment_analysis_project/"
df = spark.read.csv(path + "data/raw/text-root.csv", header=True, inferSchema=True)

In [42]:
# Hiển thị dữ liệu 10 dòng đầu
df.show(10)

In [43]:
# Hiển thị schema
df.printSchema() 

In [44]:
# Hiển thị số lượng dòng và cột
print(f"Dataset shape: ({df.count()}, {len(df.columns)})")

In [45]:
# Phân phối của cột label
df.groupBy("label").count().show()

In [46]:
# Lọc lại dữ liệu chỉ lấy cột text và label
df = df.select("text", "label")

In [47]:
# Chỉ lấy label = 0 hoặc 1
df = df.filter(df.label.isin([0, 1]))

In [48]:
# Xem lại dữ liệu 10 dòng đầu dạng pandas
df.limit(10).toPandas()

# Làm sạch dữ liệu

In [49]:
# Lọc bỏ các dòng có giá trị null trong cột text hoặc label
df = df.dropna(subset=["text", "label"])

In [50]:
# Loại bỏ các dòng trống
df = df.filter(df.text != "")

In [51]:
# Loại bỏ các dòng trùng lặp
df = df.dropDuplicates()

In [52]:
# Loại bỏ khoảng trắng thừa ở đầu và cuối dòng
from pyspark.sql.functions import trim

df = df.withColumn("text", trim(df.text))

In [53]:
# Xem lại số lượng dòng của 2 label
df.groupBy("label").count().show()

In [54]:
# Tạo WordCloud cho các câu tích cực
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Tạo WordCloud cho các câu tích cực
text = " ".join(df.filter(df.label == 1).rdd.map(lambda x: x.text).collect())
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


In [55]:
# Tạo WordCloud cho các câu tiêu cực
text = " ".join(df.filter(df.label == 0).rdd.map(lambda x: x.text).collect())
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [56]:
# Lưu dữ liệu đã xử lý
df.write.csv(path + "data/eda/", header=True, mode="overwrite")

In [57]:
# Kết thúc SparkSession
spark.stop()  # Uncomment nếu muốn dừng session